# Notebook com as funções necessárias para obter os dados de Benchmark

## Definição de imports

In [1]:
# imports de bibliotecas
import pandas as pd
import yfinance as yf
import pdb
import requests
import datetime
from dateutil.relativedelta import relativedelta


## Definições de variáveis globais

In [2]:
# URL SGS - Sistema Gerenciador de Séries Temporais
url_api_sgs_bacen = 'https://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json&dataInicial={}&dataFinal={}'

# URL IBGE - Serviço de Dados - IPCA Acumulado Anual
url_api_servico_dados_ibge = "http://servicodados.ibge.gov.br/api/v3/agregados/1737/periodos/all/variaveis/69?localidades=BR"

## Função para obtenção de retornos do índice Bovespa

In [3]:
# Os dados obtidos são a partir
def buscarRetornoBenchmarkIBOVESPA(data_inicio, data_fim):
   # O "len(df_tipo_ativos)-1" serve para que o df de Renda Fixa não seja considerado
  colecao_retornos_calculados = []
  precos_ativos = yf.download("^BVSP",data_inicio, data_fim )['Adj Close'].resample('Y').last()
  retornos_calculados = precos_ativos.pct_change()[1:]
  colecao_retornos_calculados.append(retornos_calculados)
  return colecao_retornos_calculados

## Função para obtenção do IPCA Acumulado Anual do IBGE

In [4]:
#Exemplo retorno API:
# [{'id': '69',
#   'variavel': 'IPCA - Variação acumulada no ano',
#   'unidade': '',
#   'resultados': [{'classificacoes': [],
#     'series': [{'localidade': {'id': '1',
#        'nivel': {'id': 'N1', 'nome': 'Brasil'},
#        'nome': 'Brasil'},
#       'serie': {'197912': '...',
#        '202201': '0.54',
#        '202202': '1.56',
#        '202203': '3.20',
#        '202204': '4.29',
#        '202205': '4.78',
#        '202206': '5.49',
#        '202207': '4.77',
#        '202208': '4.39',
#        '202209': '4.09',
#        '202210': '4.70',
#        '202211': '5.13',
#        '202212': '5.79',
#        '202301': '0.53',
#        '202302': '1.37',
#        '202303': '2.09',
#        '202304': '2.72',
#        '202305': '2.95',
#        '202306': '2.87',
#        '202307': '2.99'}}]}]}]

def buscarRetornoBenchmark_IPCA(data_inicio, data_fim):

  data_inicio = pd.to_datetime(data_inicio)
  data_fim = pd.to_datetime(data_fim)

  ano_inicio = data_inicio.year + 1
  ano_fim = data_fim.year
  requisicao = requests.get(url_api_servico_dados_ibge).json()
  anos_meses_medicao_ipca = requisicao[0]['resultados'][0]['series'][0]['serie']

  df = pd.DataFrame(columns = ['ano','valor'])
  #df.columns = ['ano','valor']

  for ano_mes in anos_meses_medicao_ipca:
    #print(ano_mes)
    mes=ano_mes[4:6:1]
    ano=ano_mes[0:4:1]
    if (mes=='12') & (ano != '1979'):
      # df['ano'] = ano
      # df['valor'] = anos_meses_medicao_ipca[ano_mes]
      df.loc[len(df)] = [ano, float(anos_meses_medicao_ipca[ano_mes])/100]

  #pdb.set_trace()
  data_hoje = datetime.date.today()
  ano_atual = str(data_hoje.year)
  # Verificar se existe o ano corente dentro dos registros de IPCA Acumulado
  df_resultado_IPCA_ano_corrente = df[df['ano'].str.contains(ano_atual)]

  if df_resultado_IPCA_ano_corrente.empty:
    ano_mes_resultado = list(anos_meses_medicao_ipca)[-1]
    df.loc[len(df)] = [ano_atual, float(anos_meses_medicao_ipca[ano_mes_resultado])/100]

  df_final = df[(df['ano'].astype('int') >= ano_inicio) & (df['ano'].astype('int') <= ano_fim)]
  return df_final

## Função para obtenção de retornos da SELIC a partir da API do Banco Central

In [5]:
def buscarRetornoBenchmark_SELIC(data_inicio, data_fim):
  codigo_bacen_SELIC = 432
  data_inicio = pd.to_datetime(data_inicio) + relativedelta(years=1)
  data_fim = pd.to_datetime(data_fim)
  data_inicio = data_inicio.strftime("%d/%m/%Y")
  data_fim = data_fim.strftime("%d/%m/%Y")
  df = pd.read_json(url_api_sgs_bacen.format(codigo_bacen_SELIC, data_inicio, data_fim))
  df['data'] = pd.to_datetime(df['data'], dayfirst=True)
  df.set_index('data', inplace=True)
  df =  df[:].resample('Y').last()
  df['valor'] = df['valor']/100
  return df

## Função para obtenção de retornos do Dólar a partir da API do Banco Central

In [6]:
def buscarRetornoBenchmark_Dolar(data_inicio, data_fim):
  codigo_bacen_dolar = 1
  data_inicio = pd.to_datetime(data_inicio) + relativedelta(years=1)
  data_fim = pd.to_datetime(data_fim)
  data_inicio = data_inicio.strftime("%d/%m/%Y")
  data_fim = data_fim.strftime("%d/%m/%Y")
  df = pd.read_json(url_api_sgs_bacen.format(codigo_bacen_dolar, data_inicio, data_fim))
  df['data'] = pd.to_datetime(df['data'], dayfirst=True)
  df.set_index('data', inplace=True)
  df =  df[:].resample('Y').last()
  #df['valor'] = df['valor']/100
  retornos_calculados = df.pct_change()[1:]
  return retornos_calculados

In [9]:
# data_inicio = '2010-01-01'
# data_fim = '2023-12-31'

# buscarRetornoBenchmark_SELIC(data_inicio, data_fim)

,valor
data,
2011-12-31,0.1100
2012-12-31,0.0725
2013-12-31,0.1000
2014-12-31,0.1175
2015-12-31,0.1425
2016-12-31,0.1375
2017-12-31,0.0700
2018-12-31,0.0650
2019-12-31,0.0450
